In [1]:
def has_docstring(func):
    """Check to see if the function 'func' has a docstring.

    Args:
        func (callable): A function.

    Returns:
        bool
    """
    return func.__doc__ is not None

In [3]:
def create_math_function(func_name):
    if func_name == "add":
        def add(a, b):
            return a + b
        return add
    elif func_name == "subtract":
        # Define the subtract() function
        def subtract(a, b):
            return a - b
        return subtract
    else:
        print("I don't know that one")

add = create_math_function("add")
print("5 + 2 = {}".format(add(5, 2)))

subtract = create_math_function("subtract")
print("6 - 3 = {}".format(subtract(6, 3)))

5 + 2 = 7
6 - 3 = 3


# Scope

In [5]:
x = 50

def one():
    x = 10

def two():
    global x
    x = 30

def three():
    x = 100
    print(x)

for func in [one, two, three]:
    func()
    print(x)

50
30
100
30


### Modifying variables outside local scope
Sometimes your functions will need to modify a variable that is outside of the local scope of that function. While it's generally not best practice to do so, it's still good to know-how in case you need to do it. Update these functions so they can modify variables that would usually be outside of their scope.

In [12]:
# Add a keyword that lets us update call_count from inside the function.
call_count = 0

def my_function():
  # Use a keyword that lets us update call_count 
  global call_count
#   call_count = 0
  call_count += 1
  
  print("You've called my_function() {} times!".format(
    call_count
  ))
  
for _ in range(20):
  my_function()

You've called my_function() 1 times!
You've called my_function() 2 times!
You've called my_function() 3 times!
You've called my_function() 4 times!
You've called my_function() 5 times!
You've called my_function() 6 times!
You've called my_function() 7 times!
You've called my_function() 8 times!
You've called my_function() 9 times!
You've called my_function() 10 times!
You've called my_function() 11 times!
You've called my_function() 12 times!
You've called my_function() 13 times!
You've called my_function() 14 times!
You've called my_function() 15 times!
You've called my_function() 16 times!
You've called my_function() 17 times!
You've called my_function() 18 times!
You've called my_function() 19 times!
You've called my_function() 20 times!


In [ ]:
def read_files():
    file_contents = None

    def save_contents(filename):
        # Add a keyword that lets up modify file_contentes
        nonlocal file_contents
        if file_contents is None:
            file_contents = []
        with open(filename) as fin:
            file_contents.append(fin.read())
    
    for filename in ["1984.txt", "MobyDict.txt", "CatsEye.txt"]:
        save_contents(filename)

    return file_contents

In [14]:
import random 

def wait_until_done():
    def check_is_done():
        # Add a keyword so that wait_until_done()
        # doesn't run forever
        global done
        if random.random() < 0.1:
            done = True
    
    while not done:
        check_is_done()

done = False
wait_until_done()

print("Work done? {}".format(done))

Work done? True


Stellar scoping! By adding global done in check_is_done(), you ensure that the done being referenced is the one that was set to False before wait_until_done() was called. Without this keyword, wait_until_done() would loop forever because the done = True in check_is_done() would only be changing a variable that is local to check_is_done(). Understanding what scope your variables are in will help you debug tricky situations like this one.

In [15]:
def foo():
    a = 4
    def bar():
        print(a)
    return bar

func = foo()
func()

4


In [16]:
type(func.__closure__)

tuple

In [17]:
len(func.__closure__)

1

In [18]:
func.__closure__[0].cell_contents

4

In [19]:
def return_a_func(arg1, arg2):
  def new_func():
    print('arg1 was {}'.format(arg1))
    print('arg2 was {}'.format(arg2))
  return new_func
    
my_func = return_a_func(2, 17)

print(my_func.__closure__ is not None)
print(len(my_func.__closure__) == 2)

# Get the values of the variables in the closure
closure_values = [
  my_func.__closure__[i].cell_contents for i in range(2)
]
print(closure_values == [2, 17])

True
True
True


Case closed! Your niece is relieved to see that the values she passed to return_a_func() are still accessible to the new function she returned, even after the program has left the scope of return_a_func().

Values get added to a function's closure in the order they are defined in the enclosing function (in this case, arg1 and then arg2), but only if they are used in the nested function. That is, if return_a_func() took a third argument (e.g., arg3) that wasn't used by new_func(), then it would not be captured in new_func()'s closure.

## Closures keep your values safe
You are still helping your niece understand closures. You have written the function get_new_func() that returns a nested function. The nested function call_func() calls whatever function was passed to get_new_func(). You've also written my_special_function() which simply prints a message that states that you are executing my_special_function().

You want to show your niece that no matter what you do to my_special_function() after passing it to get_new_func(), the new function still mimics the behavior of the original my_special_function() because it is in the new function's closure.

In [20]:
def my_special_function():
  print('You are running my_special_function()')
  
def get_new_func(func):
  def call_func():
    func()
  return call_func

new_func = get_new_func(my_special_function)

# Redefine my_special_function() to just print "hello"
def my_special_function():
  print("hello")

new_func()

You are running my_special_function()


In [21]:
# Show that even if you delete my_special_function(), you can still call new_func() without any problems.
# Delete my_special_function()
del my_special_function

new_func()

You are running my_special_function()


In [23]:
# Show that you still get the original message even if you overwrite my_special_function() with the new function.
# Overwrite `my_special_function` with the new function
def my_special_function():
  print('You are running my_special_function()')
  
def get_new_func(func):
  def call_func():
    func()
  return call_func

# Overwrite `my_special_function` with the new function
my_special_function = get_new_func(my_special_function)

my_special_function()

You are running my_special_function()


## Using decorator syntax
You have writtern a decorator called pring_args that prints out all of the arguments and their values any time a function that it is decorating gets called.

In [1]:
def my_function(a, b, c):
    print(a + b + c)

# Decorate my_function() with the print_args() decorator
my_function = print_args(my_function)

my_function(1, 2, 3)

NameError: name 'print_args' is not defined

In [2]:
# Decorate my_function() with the pring_args() decorator
@pring_args
def my_function(a, b, c):
    print(a + b + c)

my_function(1, 2, 3)

NameError: name 'pring_args' is not defined

## Defining a decorator
Your buddy has been working on a decorator that prints a "before" message before the decorated function is called and prints an "after" message after the decorated function is called. They are having trouble remembering how wrapping the decorated function is supposed to work. Help them out by finishing their print_before_and_after() decorator.

In [3]:
def print_before_and_after(func):
    def wrapper(*args):
        print("Before {}".format(func.__name__))
        # Call the function beging decorated with *args
        func(*args)
        print("After {}".format(func.__name__))
    # Return the nested function
    return wrapper 

@print_before_and_after
def multiply(a, b):
    print(a * b)

multiply(5, 10)

Before multiply
50
After multiply
